In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

23/04/13 20:48:58 WARN Utils: Your hostname, jakub-G3-3590 resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp3s0)
23/04/13 20:48:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 20:48:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Zadanie 3

In [17]:
from datetime import datetime
from decimal import Decimal

from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DecimalType


schema = StructType([
    StructField("AccountId", IntegerType()),
    StructField("TranDate", DateType()),
    StructField("TranAmt", DecimalType(8, 2))
])

data = [
    ( 1, '2011-01-01', 500),
    ( 1, '2011-01-15', 50),
    ( 1, '2011-01-22', 250),
    ( 1, '2011-01-24', 75),
    ( 1, '2011-01-26', 125),
    ( 1, '2011-01-28', 175),
    ( 2, '2011-01-01', 500),
    ( 2, '2011-01-15', 50),
    ( 2, '2011-01-22', 25),
    ( 2, '2011-01-23', 125),
    ( 2, '2011-01-26', 200),
    ( 2, '2011-01-29', 250),
    ( 3, '2011-01-01', 500),
    ( 3, '2011-01-15', 50 ),
    ( 3, '2011-01-22', 5000),
    ( 3, '2011-01-25', 550),
    ( 3, '2011-01-27', 95 ),
    ( 3, '2011-01-30', 2500)
]

data = list(map(lambda x: (x[0], datetime.strptime(x[1], '%Y-%m-%d').date(), Decimal(x[2])), data))
df_transactions = spark.createDataFrame(schema=schema, data=data)

In [18]:
schema = StructType([
    StructField("RowID", IntegerType()),
    StructField("FName", StringType()),
    StructField("Salary", IntegerType())
])

data = [
    (1,'George', 800),
    (2,'Sam', 950),
    (3,'Diane', 1100),
    (4,'Nicholas', 1250),
    (5,'Samuel', 1250),
    (6,'Patricia', 1300),
    (7,'Brian', 1500),
    (8,'Thomas', 1600),
    (9,'Fran', 2450),
    (10,'Debbie', 2850),
    (11,'Mark', 2975),
    (12,'James', 3000),
    (13,'Cynthia', 3000),
    (14,'Christopher', 5000)
]

df_logical = spark.createDataFrame(schema=schema, data=data)

In [25]:
from pyspark.sql.window import Window

window_ = Window.partitionBy('AccountId').orderBy("TranDate")
df_transactions.withColumn("RunTotalAmt", sum(col("TranAmt")).over(window_)).orderBy(["AccountId", "TranDate"]).show()

+---------+----------+-------+-----------+
|AccountId|  TranDate|TranAmt|RunTotalAmt|
+---------+----------+-------+-----------+
|        1|2011-01-01| 500.00|     500.00|
|        1|2011-01-15|  50.00|     550.00|
|        1|2011-01-22| 250.00|     800.00|
|        1|2011-01-24|  75.00|     875.00|
|        1|2011-01-26| 125.00|    1000.00|
|        1|2011-01-28| 175.00|    1175.00|
|        2|2011-01-01| 500.00|     500.00|
|        2|2011-01-15|  50.00|     550.00|
|        2|2011-01-22|  25.00|     575.00|
|        2|2011-01-23| 125.00|     700.00|
|        2|2011-01-26| 200.00|     900.00|
|        2|2011-01-29| 250.00|    1150.00|
|        3|2011-01-01| 500.00|     500.00|
|        3|2011-01-15|  50.00|     550.00|
|        3|2011-01-22|5000.00|    5550.00|
|        3|2011-01-25| 550.00|    6100.00|
|        3|2011-01-27|  95.00|    6195.00|
|        3|2011-01-30|2500.00|    8695.00|
+---------+----------+-------+-----------+



In [26]:
df_transactions \
  .withColumn("RunAvg", avg(col("TranAmt")).over(
    Window.partitionBy("AccountId").orderBy("TranDate")
)).withColumn("RunTranQty", count("*").over(
    Window.partitionBy("AccountId").orderBy("TranDate")
)).withColumn("RunSmallAmt", min(col("TranAmt")).over(
    Window.partitionBy("AccountId").orderBy("TranDate")
)).withColumn("RunLargeAmt", max(col("TranAmt")).over(
    Window.partitionBy("AccountId").orderBy("TranDate")
)).withColumn("RunTotalAmt", sum(col("TranAmt")).over(
    Window.partitionBy("AccountId").orderBy("TranDate")
)).orderBy(["AccountId", "TranDate"]).show()

+---------+----------+-------+-----------+----------+-----------+-----------+-----------+
|AccountId|  TranDate|TranAmt|     RunAvg|RunTranQty|RunSmallAmt|RunLargeAmt|RunTotalAmt|
+---------+----------+-------+-----------+----------+-----------+-----------+-----------+
|        1|2011-01-01| 500.00| 500.000000|         1|     500.00|     500.00|     500.00|
|        1|2011-01-15|  50.00| 275.000000|         2|      50.00|     500.00|     550.00|
|        1|2011-01-22| 250.00| 266.666667|         3|      50.00|     500.00|     800.00|
|        1|2011-01-24|  75.00| 218.750000|         4|      50.00|     500.00|     875.00|
|        1|2011-01-26| 125.00| 200.000000|         5|      50.00|     500.00|    1000.00|
|        1|2011-01-28| 175.00| 195.833333|         6|      50.00|     500.00|    1175.00|
|        2|2011-01-01| 500.00| 500.000000|         1|     500.00|     500.00|     500.00|
|        2|2011-01-15|  50.00| 275.000000|         2|      50.00|     500.00|     550.00|
|        2

In [27]:
window_ = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(Window.currentRow - 2, Window.currentRow)

In [36]:
df_transactions.withColumn("SlideAvg", avg(col("TranAmt")).over(window_ )) \
                .withColumn("SlideQty", count("*").over(window_)) \
                .withColumn("SlideMin", min(col("TranAmt")).over(window_)) \
                .withColumn("SlideMax", max(col("TranAmt")).over(window_)) \
                .withColumn("SlideTotal", sum(col("TranAmt")).over(window_)) \
                .withColumn("RN", row_number().over(
                    Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(Window.unboundedPreceding, Window.currentRow)
                    )
                ).orderBy(["AccountId", "TranDate"]).show()


+---------+----------+-------+-----------+--------+--------+--------+----------+---+
|AccountId|  TranDate|TranAmt|   SlideAvg|SlideQty|SlideMin|SlideMax|SlideTotal| RN|
+---------+----------+-------+-----------+--------+--------+--------+----------+---+
|        1|2011-01-01| 500.00| 500.000000|       1|  500.00|  500.00|    500.00|  1|
|        1|2011-01-15|  50.00| 275.000000|       2|   50.00|  500.00|    550.00|  2|
|        1|2011-01-22| 250.00| 266.666667|       3|   50.00|  500.00|    800.00|  3|
|        1|2011-01-24|  75.00| 125.000000|       3|   50.00|  250.00|    375.00|  4|
|        1|2011-01-26| 125.00| 150.000000|       3|   75.00|  250.00|    450.00|  5|
|        1|2011-01-28| 175.00| 125.000000|       3|   75.00|  175.00|    375.00|  6|
|        2|2011-01-01| 500.00| 500.000000|       1|  500.00|  500.00|    500.00|  1|
|        2|2011-01-15|  50.00| 275.000000|       2|   50.00|  500.00|    550.00|  2|
|        2|2011-01-22|  25.00| 191.666667|       3|   25.00|  500

In [37]:
window1 = Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)
window2 = Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, Window.currentRow)

df_logical.withColumn("SumByRows", sum(col("Salary")).over(window1)) \
          .withColumn("SumByRange", sum(col("Salary")).over(window2)) \
          .orderBy("RowID") \
          .show()

23/04/13 22:02:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/13 22:02:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/13 22:02:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/13 22:02:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/13 22:02:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----+-----------+------+---------+----------+
|RowID|      FName|Salary|SumByRows|SumByRange|
+-----+-----------+------+---------+----------+
|    1|     George|   800|     

# Zadanie 4

In [59]:
window_spec = Window.partitionBy("RowID").orderBy("Salary") # .rowsBetween(-2, 0)

In [60]:
df_logical.withColumn("LeadByRows", lead(col("Salary"), 2).over(window_spec)) \
    .withColumn("LagByRows", lag(col("Salary"), 2).over(window_spec)) \
    .withColumn("RowNumber", row_number().over(window_spec)) \
    .orderBy("RowID") \
    .show()

+-----+-----------+------+----------+---------+---------+
|RowID|      FName|Salary|LeadByRows|LagByRows|RowNumber|
+-----+-----------+------+----------+---------+---------+
|    1|     George|   800|      null|     null|        1|
|    2|        Sam|   950|      null|     null|        1|
|    3|      Diane|  1100|      null|     null|        1|
|    4|   Nicholas|  1250|      null|     null|        1|
|    5|     Samuel|  1250|      null|     null|        1|
|    6|   Patricia|  1300|      null|     null|        1|
|    7|      Brian|  1500|      null|     null|        1|
|    8|     Thomas|  1600|      null|     null|        1|
|    9|       Fran|  2450|      null|     null|        1|
|   10|     Debbie|  2850|      null|     null|        1|
|   11|       Mark|  2975|      null|     null|        1|
|   12|      James|  3000|      null|     null|        1|
|   13|    Cynthia|  3000|      null|     null|        1|
|   14|Christopher|  5000|      null|     null|        1|
+-----+-------